# Explore SDSS Stripe 82 RR Lyrae catalog with period-folding

This short example notebook demonstrates how to use TAPE to explore the SDSS Stripe 82 RR Lyrae catalog. We will use a Lomb–Scargle periodogram to extract periods from r-band light curves and select the RR Lyrae star with the most confident period determination. Then, we will plot the period-folded light curve for this RR Lyrae star.

In [ ]:
import matplotlib.pyplot as plt
from light_curve import Periodogram
from tape import Ensemble

In [ ]:
# Load SDSS Stripe 82 RR Lyrae catalog
ens = Ensemble(client=False).from_dataset("s82_rrlyrae")

In [ ]:
%%time

# Filter out invalid detections, "flux" denotes magnitude column
ens = ens.query("10 < flux < 25", table="source")

# Find periods using Lomb-Scargle periodogram
periodogram = Periodogram(peaks=1, nyquist=0.1, max_freq_factor=10, fast=False)

# Use r band only
df = ens.batch(periodogram, band_to_calc="r").compute()
display(df)

# Find RR Lyr with the most confient period
id = df.index[df["period_s_to_n_0"].argmax()]
period = df["period_0"].loc[id]

In [ ]:
# Plot folded light curve
ts = ens.to_timeseries(id)
COLORS = {"u": "blue", "g": "green", "r": "orange", "i": "red", "z": "purple"}
color = [COLORS[band] for band in ts.band]
plt.title(f"{id} P={period:.3f} d")
plt.gca().invert_yaxis()
plt.scatter(ts.time % period / period, ts.flux, c=color, s=7)
plt.xlim([0, 1])
plt.xlabel("Phase")
plt.ylabel("Magnitude")